In [1]:
using Interpolations
import Optim: optimize
import QuantEcon: rouwenhorst
import Roots: find_zero, Brent
import Distributions: cdf, Normal
using Random

# OECD scale
function oecd(m,n)
    # oecd equivalence scale (1 + 0.7(Adults - 1) + 0.5 kids)
    # m = `1 is married
    # n family size`
    return (1 + 0.7(m) + 0.5(n-(m + 1)))
end

# CRRA utility function
function u(c,gamma)
    Val = ((c)^(1-gamma)) / (1-gamma)
    #if Val == -Inf
    #    Val = -100000
    #end
    return Val
end

# CRRA utility function scaled
function u_hh(c,gamma, m,n)
    Val = ((c/oecd(m,n))^(1-gamma)) / (1-gamma)
    return n * Val
end

############# transfer functions
function shocks_out_prob(r,n,m, j, p, past_in, past_out)
    # expects log income
    mid_age = 22.5 + 4*j

    if r == 2 
        #val = -4.836454 - 0.10948 * n - 0.13085 *m - (0.01416632 *mid_age)  - (0.000019047 *mid_age^2) + p * 0.46988 -0.01260 - 1.06999
        #val = -5.631132 - 0.11958 * n - 0.11327 *m + (0.004895256 *mid_age)  - (0.0001556629 *mid_age^2) + p *0.47398 -0.70475 - 0.14519 # transfer of at least 200
        #val = - 2.290458 - 0.11306 * n - 0.11327 *m + (0.01672441 *mid_age)  - (0.0005318148 *mid_age^2) - past_in*0.50747 + past_out* 1.45508 +p *0.32225 -0.52712 - 1.88505  # transfer of at least 200
        val = - 2.304074 - 0.11377 * n - 0.04575 *m + (0.01669455 *mid_age)  - (0.0005308654 *mid_age^2) - past_in*0.50769 + past_out* 1.45337 +p *0.32320 -0.53053 - 1.87701 # transfer min, income max 180000
    else
        #val = -5.530618 - 0.1168 *n + 0.19154*m + (0.0519 *mid_age)  - (0.0004 *mid_age^2) + p *  0.2984 - 0.50525 - 0.13619
        #val = -5.852817 - 0.11739 *n + 0.04640*m + (0.0559236683 *mid_age)  - (0.0004568606 *mid_age^2) + p *  0.31552 - 0.49947 - 0.11855 # transfer of at least 200
       # val = -4.695883 - 0.09430 *n + 0.18568*m + (0.0394403367 *mid_age)  - (0.0003222023 *mid_age^2) -past_in*0.08426 + past_out*1.73097 + p * 0.24772 - 1.71297 + 1.04308# transfer of at least 200
        val = -4.463962 - 0.09363 *n + 0.18690*m + (0.0397062151 *mid_age)  - (0.0003243744 *mid_age^2) -past_in*0.08483 + past_out*1.73401 + p * 0.24322 - 1.71271 + 1.04578 # # transfer min, income max 180000

    end

    return cdf(Normal(), val)
end

function transfers_out_amount(r,n,m, j, p)
    mid_age = 22.5 + 4*j

    if r == 2
        #return exp(-1.651336 - 0.186676*n - 0.099164*m + (0.029 *mid_age) - (0.0002315 *mid_age^2) +log(p) * 0.7149 + 0.149022 + 0.2810)
        return exp(-0.9451237 - 0.12296*n - 0.23850*m + (0.0419915484 *mid_age) - (0.0003287593 *mid_age^2) +log(p) * 0.62658 + 0.19965 + 0.17305) # transfer of at least 200, income max 180000
    else
        #return exp(-4.155144 -0.140 *n - 0.03*m + (0.057 *mid_age)  - (0.0004 *mid_age^2) +log(p) * 0.845 + 0.885885 + 0.197426)
        #return exp(-3.138253 -0.13747 *n - 0.05566*m + (0.0442593370 *mid_age)  - (0.0003160808 *mid_age^2) +log(p) * 0.79922 + 0.70891 + 0.21847) # transfer of at least 200
        return exp(-3.00629 -0.134370 *n - 0.054113*m + (0.0441335028 *mid_age)  - (0.0003151821 *mid_age^2) +log(p) * 0.788086 + 0.714118 + 0.212124 ) # transfer of at least 200, income max 180000
    end

end

function shocks_in_prob(r,n,m,j,p, past_in, past_out)
    # expects log income
    mid_age = 22.5 + 4*j

    if r == 2
       # val = 4.50985 - 0.05770*n - 0.10076*m  - (0.04776 *mid_age)  + (0.0003 *mid_age^2) - p * 0.26408 - 0.63672 - 0.31932
        #val = -0.9319242 - 0.06199*n - 0.07126*m  - (0.0405899612 *mid_age)  + (0.0002587213 *mid_age^2) - p * 0.22240 + 4.07789 - 0.32435 # transfer of at least 200
        val = 1.458929 - 0.09941*n - 0.15475*m  - (0.1211193934 *mid_age)  + (0.0007720176 *mid_age^2) +past_in* 1.46933 - past_out*0.06697 - p * 0.18051 + 2.96273 - 0.27434 # transfer of at least 200
    else
        #val = 1.143652 + 0.01461*n - 0.16394*m - (0.042635 *mid_age) + (0.00025999 *mid_age^2) - p * 0.33311 + 3.01575 + 0.22755
       # val = 0.8746951 + 0.01347*n - 0.15700*m - (0.0417002145 *mid_age) + (0.0002550053 *mid_age^2) - p * 0.31443 + 3.01634 + 0.22581 # transfer of at least 200
       # val = 3.603418 + 0.04395*n + 0.06475*m - (0.1094734863 *mid_age) + (0.0006694526 *mid_age^2) + past_in*1.59049 - past_out*0.32699 - p * 0.30980 + 1.82982 + 0.07395 # transfer of at least 200
        val = 3.600208 -0.04262*n + 0.06188*m - (0.1096650013 *mid_age) + (0.0006706238 *mid_age^2) + past_in*1.58966 - past_out*0.32242 - p * 0.30893 + 1.82198 + 0.07752 # transfer of at least 200, income max 180000
    end
    return cdf(Normal(), val)
end

function transfers_in_amount(r,n,m,j,p)
    mid_age = 22.5 + 4*j

    if r == 2
        #return exp(2.298028 - 0.06135*n - 0.10296*m + (0.095129 *mid_age)  - (0.0009649053*mid_age^2) + log(p) * 0.18970 + 0.58675 -0.03346)
        return exp(3.128561 - 0.06088*n - 0.17337*m + (0.098899763 *mid_age)  - (0.001003151*mid_age^2) + log(p) * 0.11010 + 0.89794 -0.13826) # transfers of at least 200
    else
        #return exp(2.409782 - 0.07685*n + 0.159522 *m + (0.0494429521 *mid_age)  - (0.0004916551 *mid_age^2) + log(p) *0.345546 + 0.371002 + 0.525305)
        #return exp(3.054937 - 0.07382*n + 0.12053 *m + (0.0509047783 *mid_age)  - (0.0005061923  *mid_age^2) + log(p) *0.30968 +0.44749 + 0.30191) # transfers of at least 200
        return exp(3.041345 - 0.07338*n + 0.11436 *m + (0.0518311507 *mid_age)  - (0.0005127471  *mid_age^2) + log(p) *0.30850 +0.31655 + 0.44215) # transfer of at least 200, income max 180000

    end
end

# deterministic component of log income
function g(r,j,n)
    mid_age = (22.5 + 4*j)

    if r == 2
        g = 4.469 + 0.4817*mid_age - 0.01037*mid_age^2 + 0.00007375*mid_age^3 + 0.06854*n -0.08079
    else
        #g = 9.676 + 0.1203*mid_age - 0.001434*(mid_age^2)  + 0.000002434*(mid_age^3) + 0.08323*n + 0.05215 # no max 
        #g = 9.476 +  0.1381*mid_age - 0.001912*(mid_age^2)  + 0.000006347*(mid_age^3) + 0.0788*n + 0.04844 # max 2000000
        #g = 9.562 +  0.1323*mid_age - 0.001783*(mid_age^2)  + 0.000005397*(mid_age^3) + 0.07839*n + 0.04567 # max 1500000
        g = 9.509 +  0.1356*mid_age - 0.001852*(mid_age^2)  + 0.000005373*(mid_age^3) + 0.07886*n + 0.04664 # max 1800000
    end
    return g
end

# government transfer
function GT(j,y,a, m, n, cw, co, tax_a, rate)
    # consumption floor scaled by family size
    s(m,n) = j < 5 ? oecd(m,n) : oecd(m,m+1)

    # j age/period, y income/pension, a asset, tau_y labor income tax
    if j <= 9 
        # transfer for worker
        gt = max(0, cw * s(m,n) - (y - tax_y(y,m) + (1 + rate*(1-tax_a))*a ))
    else
        # trasnfer for retiree
        gt = max(0, co * s(m,n) - (y + (1 + rate*(1-tax_a))*a ))
    end

    return gt 
end

function income(r, j, n, z)
    if j <= 9
        # log income
        logy = g(r,j,n) + z

        # income
        y = exp(logy)

    else 
        # z is interpreted as the last income shock from working period
        logy = g(r,9,n) + z

        y = exp(logy) * 0.4
    end 
    return y
end

# end-of-period resources
function Y_solve_inputs(inc_input, t_in_amount, t_out_amount,j,m,n,shock_in, shock_out, z, a, cw, co, tax_a, rate)

    # income with shock or pension 
    y = income(inc_input, j, n, z)

    # transfers
    gov_trf = GT(j,y,a,  m, n, cw, co, tax_a, rate)
    bft_y = y + rate*a + gov_trf
    t_in = transfers_in_amount(t_in_amount,n,m,j,bft_y)
    t_out = transfers_out_amount(t_out_amount,n,m,j,bft_y)

    after_shocks_y = (1 + rate*(1-tax_a))*a + y - tax_y(y,m) + shock_in*t_in - shock_out*t_out + gov_trf

    return max(0, after_shocks_y)
end

function tax_y(y, m)
    div_y = y/4
    
    if m == 1
        lambda = 0.08
        tau_y = 0.115
    else 
        lambda = 0.07
        tau_y = 0.07
    end
    peryear = div_y - (1-lambda)*div_y^(1-tau_y)
    return peryear * 4
end

function y_plus_asset_inc(y, a, rate)
    return y + rate* a
end
## add (1+r)a to transfer function so that it is not of just labor income esp since total fam income include assets

y_plus_asset_inc (generic function with 1 method)

In [ ]:
g.(2, 1:10, 3 )

In [ ]:
transfers_in_amount.(1,3,1,1:10,exp.(g.(1, 1:10, 3 )))

In [ ]:
transfers_out_amount.(2,3,1,1:10,exp.(g.(2, 1:10, 3 )))

In [3]:
function polyexpandgrid(n, a, b, theta)
    # n is the number of points in the grid
    # grid from [a,b]
    # theta is the expansion factor

    # step 1 create an equally spaced grid
    grid1 = collect(LinRange(0,1,n))
    # step 2 shift and expand the grid
    # xj = a + (b-a)xj^theta
    grid2 = repeat([0.0],n) # place holder
    for i in 1:n
    grid2[i] = a + (b-a)*grid1[i]^theta
    end
    return grid2
end

## Step 0: Set n = 0. Construct a grid for tomorrows savings and today's shocks 
function Markov(zpnts, r)
    if r == 1
        MC =  rouwenhorst(zpnts, 0.872804, 0.4245223, 0)  # max 1800000
    elseif r == 2
        MC =  rouwenhorst(zpnts, 0.86144, 0.4434178, 0) 
    end

    return MC.p, MC.state_values
end

Markov (generic function with 1 method)

In [4]:
# Model Set Up (Generates Model Parameters)
function model_gen(; beta = 0.96, # discount factor
                     rate = 0.04, # interest rate
                     tax_a = 0.2, # capital income tax
                     gamma = 1.5, # risk aversion 
                     co = 19484.96, # consumption floor
                     cw = 19484.96,
                     apnts = 250,
                     zpnts = 10)
    
    # asset grid
    asset_grid = [0; polyexpandgrid(apnts-1, 1500, 3000000, 1.3)] 

    return(; beta, rate, tax_a, gamma, co, cw, zpnts, apnts, asset_grid)
end

# Income parameters and grids
function income_transfers(inc_input, asset_grid, n, m, t_in_prob, t_out_prob, model)
    (; beta, rate, tax_a, gamma, co, cw, zpnts, apnts, asset_grid) = model
    Pimat, zgrid = Markov(zpnts, inc_input)

    precomputed_y = exp.([g(inc_input, j, j <= 4 ? n : m+1) .+ z for j in 1:9, z in zgrid ])
    precompute_y_plus_assets  = y_plus_asset_inc.(precomputed_y, reshape(asset_grid, 1, 1, :), rate)
    precomputed_gov_trf = GT.(reshape(1:9, :, 1, 1), precomputed_y, reshape(asset_grid, 1, 1, :), m, reshape([j <= 4 ? n : m+1 for j in 1:9], :, 1, 1), cw, co, tax_a, rate)
    precomputed_bft_y = precompute_y_plus_assets .+ precomputed_gov_trf

    precomputed_p = repeat(precomputed_y[9, :]' .* 0.4, 6, 1)
    precomputed_gov_trf_ret = [GT(j, precomputed_p[i_z], a, m, m+1, cw, co, tax_a, rate) for j in 10:15, i_z in 1:zpnts, a in asset_grid]
    precomputed_bft_y_ret = y_plus_asset_inc.(precomputed_p, reshape(asset_grid, 1,1, :), rate) .+ precomputed_gov_trf_ret

    shock_expectation_mat = zeros(1:15,zpnts, apnts, 2, 2, 2, 2)
    # given states and past shocks, probability of receiving a shock in or shock out
    for past_in in 0:1
        for past_out in 0:1
            precomputed_shocks_in = shocks_in_prob.(t_in_prob, reshape([j <= 4 ? n : m+1 for j in 1:9], :, 1, 1),m,reshape(1:9, :, 1, 1),log.(precomputed_bft_y),past_in,past_out)
            precomputed_shocks_out = shocks_out_prob.(t_in_prob, reshape([j <= 4 ? n : m+1 for j in 1:9], :, 1, 1),m,reshape(1:9, :, 1, 1),log.(precomputed_bft_y),past_in,past_out)
            precomputed_shocks_in_ret= shocks_in_prob.(t_out_prob, m+1,m,reshape(10:15, :, 1, 1),log.(precomputed_bft_y_ret), past_in,past_out)
            precomputed_shocks_out_ret= shocks_out_prob.(t_out_prob, m+1,m,reshape(10:15, :, 1, 1),log.(precomputed_bft_y_ret), past_in,past_out)

            for j in 1:9
                shock_in_val = reshape(precomputed_shocks_in[j, :, :], zpnts,  apnts, 1, 1)
                shock_out_val = reshape(precomputed_shocks_out[j, :, :], zpnts,  apnts, 1, 1)

                shock_expectation_mat[j,:,:,1,1,past_in+1, past_out+1] = (1 .- shock_in_val) .* (1 .- shock_out_val)
                shock_expectation_mat[j,:, :, 1, 2,past_in+1, past_out+1] = (1 .- shock_in_val) .* shock_out_val
                shock_expectation_mat[j,:, :, 2, 1,past_in+1, past_out+1] = shock_in_val .* (1 .- shock_out_val) 
                shock_expectation_mat[j,:, :, 2, 2,past_in+1, past_out+1] = shock_in_val .* shock_out_val 
            end
            for j in 1:6

                shock_in_val = reshape(precomputed_shocks_in_ret[j, :, :], zpnts,  apnts, 1, 1)
                shock_out_val = reshape(precomputed_shocks_out_ret[j, :, :], zpnts,  apnts, 1, 1)
        
                shock_expectation_mat[j+9, :, :, 1, 1,past_in+1, past_out+1] = (1 .- shock_in_val) .* (1 .- shock_out_val)
                shock_expectation_mat[j+9,:, :, 1, 2,past_in+1, past_out+1] = (1 .- shock_in_val) .* shock_out_val
                shock_expectation_mat[j+9,:, :, 2, 1,past_in+1, past_out+1] = shock_in_val .* (1 .- shock_out_val) 
                shock_expectation_mat[j+9,:, :, 2, 2,past_in+1, past_out+1] = shock_in_val .* shock_out_val 
            end

        end
    end

 
    return(; Pimat, zgrid, shock_expectation_mat)
end

function inputs(inc_input, t_in_amount, t_out_amount, t_in_prob, t_out_prob, surv_input, m, n, asset_grid, model)

    (; beta, rate, tax_a, gamma, co, cw, zpnts, apnts, asset_grid) = model

    (; Pimat, zgrid, shock_expectation_mat) = income_transfers(inc_input, asset_grid, n, m, t_in_prob, t_out_prob, model)
    
    surv_risk = survival_prob[:,surv_input]

    net_resources = [Y_solve_inputs(inc_input, t_in_amount, t_out_amount, j, m, j <= 4 ? n : m+1, shock_in, shock_out, z, a,  cw, co, tax_a, rate)
                        for j in 1:15, z in zgrid, a in asset_grid, shock_in in 0:1, shock_out in 0:1]
    
    return (;inc_input, t_in_amount, t_out_amount, t_in_prob, t_out_prob, m, n, Pimat, zgrid, shock_expectation_mat, net_resources, surv_risk)
end

inputs (generic function with 1 method)

In [10]:
function T_itp(VF, WF, model_param, inputs, m, n)

    # parameters
    (; beta, rate, tax_a, gamma, co, cw, zpnts, apnts, asset_grid) = model_param
    (;inc_input, t_in_amount, t_out_amount, t_in_prob, t_out_prob, m, n, Pimat, zgrid, shock_expectation_mat,net_resources, surv_risk) = inputs
    shock_range = 0:1
    VF_new = similar(VF)
    PF_new = similar(VF)
    WF_new = similar(WF)
    WPF_new = similar(WF)

    VF_itp = interpolate((1:9, zgrid, asset_grid, shock_range, shock_range, shock_range, shock_range), VF, Gridded(Linear()))
    VF_itp = extrapolate(VF_itp, Flat())
    WF_itp = interpolate((10:15, zgrid, asset_grid, shock_range, shock_range, shock_range, shock_range), WF, Gridded(Linear()))
    WF_itp = extrapolate(WF_itp, Flat())
    shock_exp_itp = interpolate((1:15, zgrid, asset_grid, shock_range, shock_range, shock_range, shock_range), shock_expectation_mat, Gridded(Linear()))
    shock_exp_itp = extrapolate(shock_exp_itp, Flat())

    n_size = [j <= 4 ? n : m+1 for j in 1:9]

    Threads.@threads for idx in CartesianIndices((1:9, 1:length(zgrid), 1:length(asset_grid), 1:2, 1:2, 1:2, 1:2))
        j, i_z, i_a, shock_in, shock_out, past_in, past_out = Tuple(idx)
        z = zgrid[i_z]
        a = asset_grid[i_a]
        net_resources_VF = net_resources[j, i_z, i_a, shock_in, shock_out]

        # update past transfer indicator based on current shocks
        if past_in == 2 || shock_in == 2
            next_in = 1
        else 
            next_in = 0
        end
        if past_out == 2 || shock_out == 2
            next_out = 1
        else
            next_out = 0
        end

        if j < 9
            obj_func = ap1 -> begin
                vf_interp = VF_itp[j+1, zgrid, ap1, shock_range, shock_range, next_in, next_out]
                shock_interp = shock_exp_itp[j+1, zgrid, ap1, shock_range, shock_range,next_in, next_out]
                Pimat_row = @view Pimat[i_z, :]
                sum_term =  transpose(sum(vf_interp .* shock_interp, dims = (2,3))[:]) * Pimat_row

                -(u_hh(net_resources_VF - ap1, gamma, m, n_size[j]) + beta * sum_term)
            end

            #obj_func = ap1 -> -(u_hh(net_resources_VF - ap1, gamma, m, n_size[j]) +
            #                    beta * sum(VF_itp[j+1, zgrid, ap1, shock_range, shock_range] .* 
            #                               shock_exp_itp[j+1, zgrid, ap1, shock_range, shock_range] .* Pimat[i_z, :]))
        else
            obj_func = ap1 -> begin
                wf_interp = WF_itp[10, z, ap1, shock_range, shock_range, next_in, next_out]
                shock_interp = shock_exp_itp[10, z, ap1, shock_range, shock_range, next_in, next_out]
                sum_term = sum(wf_interp .* shock_interp)

                -(u_hh(net_resources_VF - ap1, gamma, m, m+1) + beta * sum_term)
            end
            #obj_func = ap1 -> -(u_hh(net_resources_VF - ap1, gamma, m, m+1) +
            #                    beta * sum(WF_itp[10, z, ap1, shock_range, shock_range] .* 
            #                               shock_exp_itp[10, z, ap1, shock_range, shock_range]))
        end

        result = optimize(obj_func, 0.5, net_resources_VF)
        @inbounds VF_new[idx] = -result.minimum
        @inbounds PF_new[idx] = result.minimizer

        if j < 6
            net_resources_WF = net_resources[j+9, i_z, i_a, shock_in, shock_out]

            obj_func_w = ap1 -> begin
                wf_interp = WF_itp[j+10, z, ap1, shock_range, shock_range, next_in, next_out]
                shock_interp = shock_exp_itp[j+10, z, ap1, shock_range, shock_range, next_in, next_out]
                sum_term = sum(wf_interp .* shock_interp) 

                -(u_hh(net_resources_VF - ap1, gamma, m, m+1) + beta * sum_term * surv_risk[j+10])
            end

            #obj_func_w = ap1 -> -(u_hh(net_resources_WF - ap1, gamma, m, m+1) +
            #beta * sum(WF_itp[j+10, z, ap1, shock_range, shock_range] .* 
            #           shock_exp_itp[j+10, z, ap1, shock_range, shock_range]) * surv_risk[j+10])

            result_w = optimize(obj_func_w, 0.5, net_resources_WF)
            @inbounds WF_new[idx] = -result_w.minimum
            @inbounds WPF_new[idx] = result_w.minimizer
        elseif j == 6
            net_resources_WF = net_resources[15, i_z, i_a, shock_in, shock_out]

            @inbounds WF_new[idx] = u_hh(net_resources_WF, gamma, m, m+1)
            @inbounds WPF_new[idx] = 0
        end
        
    end
    return (; VF_new, PF_new, WF_new, WPF_new)

end

T_itp (generic function with 1 method)

In [ ]:
using Base.Threads
# Bellman Operator
function T_discrete(VF, WF, model_param, inputs, m, n)

    # parameters
    (; beta, rate, tax_a, gamma, co, cw, zpnts, apnts, asset_grid) = model_param
    (;inc_input, t_in_amount, t_out_amount, t_in_prob, t_out_prob, m, n, Pimat, zgrid, shock_expectation_mat, net_resources, surv_risk) = inputs

    VF_new = similar(VF)
    PF_new = similar(VF)
    WF_new = similar(WF)
    WPF_new = similar(WF)

    asset_grid_reshaped = reshape(asset_grid, 1, 1, 1, 1, apnts)

    for j in 1:9

        c_VF = net_resources[j, :, :, :, :] .- asset_grid_reshaped

        if j < 9
            VF_shocks = sum(VF[j+1, :, :, :, :] .* shock_expectation_mat[j+1, :,:,:,:], dims= (3,4))[:,:,1,1]

            EV = zeros(size(VF_shocks))
            for i_z in 1:zpnts, ap1 in 1:apnts
                EV[i_z, ap1] = sum(VF_shocks[:,ap1] .* Pimat[i_z,:])
            end
        else
            EV = sum(WF[1, :, :, :, :] .* shock_expectation_mat[j+1, :,:,:,:], dims= (3,4))[:,:,1,1]
    
        end

        EV_expand = reshape(EV, size(EV, 1), 1, 1, 1, size(EV, 2)) .* ones(1, size(c_VF, 2), size(c_VF, 3), size(c_VF, 4), 1)

        # Compute value function
        val_VF = fill(-10^8.0, zpnts, apnts, 2, 2, apnts)
        val_VF[c_VF .> 0] = u_hh.(c_VF[c_VF .> 0], gamma, m, j <= 4 ? n : m+1) .+ beta .* EV_expand[c_VF .> 0]

        # Update VF and PF
        max_vals, max_idxs = findmax(val_VF, dims=5)
        idx = map(ci -> ci[5], max_idxs)
        VF_new[j, :, :, :, :] = max_vals[:,:,:,:,1]
        PF_new[j, :, :, :, :] = asset_grid[idx]

        if j < 6
            c_WF = net_resources[j+9, :, :, :, :] .- reshape(asset_grid, 1, 1, 1, 1, apnts)
    
            EV_W = sum(WF[j+1, :, :, :, :] .* shock_expectation_mat[j+10, :,:,:,:], dims= (3,4))[:,:,1,1] .* surv_risk[j+10]
            EV_expand_W = reshape(EV_W, size(EV_W, 1), 1, 1, 1, size(EV_W, 2)) .* ones(1, size(c_WF, 2), size(c_WF, 3), size(c_WF, 4), 1)
    
            val_WF = fill(-10^8.0, zpnts, apnts, 2, 2, apnts)
            val_WF[c_WF .> 0] = u_hh.(c_WF[c_WF .> 0], gamma, m, m+1) .+ survival_prob[j, r] .* beta .* EV_expand_W[c_WF .> 0]
    
            max_vals_W, max_idxs_W = findmax(val_WF, dims=5)
            idx = map(ci -> ci[5], max_idxs_W)
            WF_new[j, :, :, :, :] = max_vals_W[:,:,:,:,1]
            WPF_new[j, :, :, :, :] = asset_grid[idx]
        elseif j == 6    
            WF_new[j, :, :, :, :] = u_hh.(net_resources[j+9, :, :, :, :], gamma, m, m+1)
            WPF_new[j, :, :, :, :] .= 0
        end
    end 

    return (; VF_new, PF_new, WF_new,WPF_new )
end

In [6]:
# VFI 
function vfi_inputs(model, inc_input, t_in_amount, t_out_amount, t_in_prob, t_out_prob, surv_input, m, n, use_interp;
                    tol = 10^-5,
                    max_iters = 15000)

    (; beta, rate, tax_a, gamma, co, cw, zpnts, apnts, asset_grid) = model

    mod_inputs = inputs(inc_input, t_in_amount, t_out_amount, t_in_prob, t_out_prob, surv_input, m, n, asset_grid, model)

    (; inc_input, t_in_amount, t_out_amount, t_in_prob, t_out_prob, m, n, Pimat, zgrid, shock_expectation_mat, net_resources, surv_risk) = mod_inputs

    # Initiate
    VF = get_value(value_functions, (inc_input, t_in_amount, t_out_amount, t_in_prob, t_out_prob,surv_input,n,m))
    WF = get_value(value_functions_w, (inc_input, t_in_amount, t_out_amount, t_in_prob, t_out_prob,surv_input,n,m))

    if VF == false
        # Initiate
        VF = ones(9,zpnts, apnts,  2, 2, 2, 2) # Initial guess for value function
        WF = ones(6,zpnts, apnts, 2, 2, 2, 2) # initial guess for value function

        for j in 1:9
            for l in 1:2
                VF[j, :, :, :, :, l, l] = u_hh.(Y_solve_inputs.(inc_input, t_in_amount, t_out_amount,j, m, j <= 4 ? n : m+1, 
                reshape([0,1], 1, 1, 1, 2), reshape([0,1], 1, 1, 1, 1, 2), zgrid, asset_grid',  cw, co, tax_a, rate), gamma, m, j <= 4 ? n : m+1)
    
            end    
        end
    
        for j in 1:5
            for l in 1:2
                WF[j, :, :, :, :, l, l] = u_hh.(Y_solve_inputs.(inc_input, t_in_amount, t_out_amount, j+9, m, m+1, 
                reshape([0,1], 1, 1, 1, 2), reshape([0,1], 1, 1, 1, 1, 2), zgrid, asset_grid',  cw, co, tax_a, rate), gamma, m, m+1)
            end
        end
    end
    
    iters  = 0

    # Iterate on Bellman's equation
    while(iters < max_iters)
        iters += 1
        #println(iters)

        # Apply Bellman operator
        if use_interp == true
            (; VF_new, PF_new, WF_new, WPF_new) = T_itp(VF, WF, model,mod_inputs, m, n)
        else
            (; VF_new, PF_new, WF_new, WPF_new) = T_discrete_2(VF, WF, model,mod_inputs, m, n)
        end

        # apply MacQueen-Porteus bounds acceleration method every 10 iterations
        if iters % 25 == 0
            clower = (beta/(1-beta)) * minimum(abs.(VF .- VF_new))
            cupper = (beta/(1-beta)) * maximum(abs.(VF .- VF_new))

            clowerW = (beta/(1-beta)) * minimum(abs.(WF .- WF_new))
            cupperW = (beta/(1-beta)) * maximum(abs.(WF .- WF_new))

            if cupper-clower <tol && cupperW-clowerW <tol
                print("Solution found")
                VF = copy(VF_new ) .+ ((cupper+clower)/2)
                WF = copy(WF_new ) .+ ((cupperW+clowerW)/2)
                return VF, PF_new, WF, WPF_new
            end
        end
    
        distVF = maximum(abs.(VF .- VF_new))
        distWF = maximum(abs.(WF .- WF_new))
        dist = max(distVF,distWF )
        if (dist < tol)
            print("Solution found")
            return (; VF, PF_new, WF, WPF_new)
        end

        # Update Value Functions
        VF = copy(VF_new)
        WF = copy(WF_new)
    end

    if iters == max_iters
        print("Max Iterations Reached")
        return VF, PF_new, WF, WPF_new
    end

end

vfi_inputs (generic function with 1 method)

In [ ]:
model = model_gen()
inc_input= 2
t_in_amount = 2
t_out_amount = 2
t_in_prob = 2
t_out_prob = 2 
surv_input = 2
m = 1
n = 3
(; VF, PF_new, WF, WPF_new) = vfi_inputs(model, inc_input, t_in_amount, t_out_amount, t_in_prob, t_out_prob, surv_input, m, n, true)

In [ ]:
model = model_gen()
inc_input= 2
t_in_amount = 2
t_out_amount = 2
t_in_prob = 2
t_out_prob = 2 
surv_input = 2
m = 1
n = 3
(; VF, PF_new, WF, WPF_new) = vfi_inputs(model, inc_input, t_in_amount, t_out_amount, t_in_prob, t_out_prob, surv_input, m, n, true)

In [7]:
function store_pf(inc_input, t_in_amount, t_out_amount, t_in_prob, t_out_prob,surv_input, n,m,PF, dict)
    key = (inc_input, t_in_amount, t_out_amount, t_in_prob, t_out_prob,surv_input,n,m)
    dict[key] = PF
    return nothing 
end

store_pf (generic function with 1 method)

In [ ]:
#policy_functions = Dict()
#value_functions = Dict()
#policy_functions_w = Dict()
#value_functions_w = Dict()
function store_pf(inc_input, t_in_amount, t_out_amount, t_in_prob, t_out_prob,surv_input, n,m,PF, dict)
    key = (inc_input, t_in_amount, t_out_amount, t_in_prob, t_out_prob,surv_input,n,m)
    dict[key] = PF
    return nothing 
end

model = model_gen()
input_range = 1:2

# 2206 minutes
# next time use old version as starting point
@inbounds for r in 1:2, n in 1:6, m in 0:1

    (; VF, PF_new, WF, WPF_new) = vfi_inputs(model, r, r, r, r, r,r, m, n, true)

    store_pf(r, r, r, r, r,r, n,m,PF_new, policy_functions )
    store_pf(r, r, r, r, r,r, n,m,WPF_new, policy_functions_w)

    if m == 1
        (; VF, PF_new, WF, WPF_new) = vfi_inputs(model, r, r, r, r, r,r, m, 7, true)

        store_pf(r, r, r, r, r,r, 7,m,PF_new, policy_functions )
        store_pf(r, r, r, r, r,r, 7,m,WPF_new, policy_functions_w)
    end

    # counterfactuals
    if r == 2
        # vfi_inputs(model, inc_input, t_in_amount, t_out_amount, t_in_prob, t_out_prob,surv_input, m, n, true)
        # income
        (; VF, PF_new, WF, WPF_new) = vfi_inputs(model, 1, r, r, r, r,r, m, n, true)

        store_pf(1, r, r, r, r,r, n,m,PF_new, policy_functions )
        store_pf(1, r, r, r, r,r, n,m,WPF_new, policy_functions_w)
        store_pf(1, r, r, r, r,r, n,m,VF, value_functions )
        store_pf(1, r, r, r, r,r, n,m,WF, value_functions_w)

        # transfer in
        (; VF, PF_new, WF, WPF_new) = vfi_inputs(model, r, 1, r, 1, r,r, m, n, true)

        store_pf(r, 1, r, 1, r,r, n,m,PF_new, policy_functions )
        store_pf(r, 1, r, 1, r,r, n,m,WPF_new, policy_functions_w)
        store_pf(r, 1, r, 1, r,r, n,m,VF, value_functions )
        store_pf(r, 1, r, 1, r,r, n,m,WF, value_functions_w)
        # transfer in amount only 
        (; VF, PF_new, WF, WPF_new) = vfi_inputs(model, r, 1, r, r, r,r, m, n, true)

        store_pf(r, 1, r, r, r,r, n,m,PF_new, policy_functions )
        store_pf(r, 1, r, r, r,r, n,m,WPF_new, policy_functions_w)
        store_pf(r, 1, r, r, r,r, n,m,VF, value_functions )
        store_pf(r, 1, r, r, r,r, n,m,WF, value_functions_w)
                #=

        # transfer in probability only
        (; VF, PF_new, WF, WPF_new) = vfi_inputs(model, r, r, r, 1, r,r, m, n, true)

        store_pf(r, r, r, 1, r,r, n,m,PF_new, policy_functions )
        store_pf(r, r, r, 1, r,r, n,m,WPF_new, policy_functions_w)
        store_pf(r, r, r, 1, r,r, n,m,VF, value_functions )
        store_pf(r, r, r, 1, r,r, n,m,WF, value_functions_w)

        # survival
        (; VF, PF_new, WF, WPF_new) = vfi_inputs(model, r, r, r, r, r,1, m, n, true)

        store_pf(r, r, r, r, r,1, n,m,PF_new, policy_functions )
        store_pf(r, r, r, r, r,1, n,m,WPF_new, policy_functions_w)
        store_pf(r, r, r, r, r,1, n,m,VF, value_functions )
        store_pf(r, r, r, r, r,1, n,m,WF, value_functions_w)

        # transfer out
        (; VF, PF_new, WF, WPF_new) = vfi_inputs(model, r, r, 1, r, 1,r, m, n, true)

        store_pf(r, r, 1, r, 1,r, n,m,PF_new, policy_functions )
        store_pf(r, r, 1, r, 1,r, n,m,WPF_new, policy_functions_w)
        store_pf(r, r, 1, r, 1,r, n,m,VF, value_functions )
        store_pf(r, r, 1, r, 1,r, n,m,WF, value_functions_w)
        # transfer out probability only
        (; VF, PF_new, WF, WPF_new) = vfi_inputs(model, r, r, r, r, 1,r, m, n, true)

        store_pf(r, r, r, r, 1,r, n,m,PF_new, policy_functions )
        store_pf(r, r, r, r, 1,r, n,m,WPF_new, policy_functions_w)
        store_pf(r, r, r, r, 1,r, n,m,VF, value_functions )
        store_pf(r, r, r, r, 1,r, n,m,WF, value_functions_w)
        # transfer out amount only
        (; VF, PF_new, WF, WPF_new) = vfi_inputs(model, r, r, 1, r, r,r, m, n, true)

        store_pf(r, r, 1, r, r,r, n,m,PF_new, policy_functions )
        store_pf(r, r, 1, r, r,r, n,m,WPF_new, policy_functions_w)
        store_pf(r, r, 1, r, r,r, n,m,VF, value_functions )
        store_pf(r, r, 1, r, r,r, n,m,WF, value_functions_w)
       =#

        # all transfers
        (; VF, PF_new, WF, WPF_new) = vfi_inputs(model, r, 1, 1, 1, 1,r, m, n, true)

        store_pf(r, 1, 1, 1, 1,r, n,m,PF_new, policy_functions )
        store_pf(r, 1, 1, 1, 1,r, n,m,WPF_new, policy_functions_w)
        store_pf(r, 1, 1, 1, 1,r, n,m,VF, value_functions )
        store_pf(r, 1, 1, 1, 1,r, n,m,WF, value_functions_w)

        if m == 1
            # income
            (; VF, PF_new, WF, WPF_new) = vfi_inputs(model, 1, r, r, r, r,r, m, 7, true)

            store_pf(1, r, r, r, r,r, 7,m,PF_new, policy_functions )
            store_pf(1, r, r, r, r,r, 7,m,WPF_new, policy_functions_w)
            store_pf(1, r, r, r, r,r, 7,m,VF, value_functions )
            store_pf(1, r, r, r, r,r, 7,m,WF, value_functions_w)
    
            # transfer in
            (; VF, PF_new, WF, WPF_new) = vfi_inputs(model, r, 1, r, 1, r,r, m, 7, true)
    
            store_pf(r, 1, r, 1, r,r, 7,m,PF_new, policy_functions )
            store_pf(r, 1, r, 1, r,r, 7,m,WPF_new, policy_functions_w)
            store_pf(r, 1, r, 1, r,r, 7,m,VF, value_functions )
            store_pf(r, 1, r, 1, r,r, 7,m,WF, value_functions_w)
            # transfer in amount only 
            (; VF, PF_new, WF, WPF_new) = vfi_inputs(model, r, 1, r, r, r,r, m, 7, true)
    
            store_pf(r, 1, r, r, r,r, 7,m,PF_new, policy_functions )
            store_pf(r, 1, r, r, r,r, 7,m,WPF_new, policy_functions_w)
            store_pf(r, 1, r, r, r,r, 7,m,VF, value_functions )
            store_pf(r, 1, r, r, r,r, 7,m,WF, value_functions_w)
                    
            # all transfers
            (; VF, PF_new, WF, WPF_new) = vfi_inputs(model, r, 1, 1, 1, 1,r, m, 7, true)

            store_pf(r, 1, 1, 1, 1,r, 7,m,PF_new, policy_functions )
            store_pf(r, 1, 1, 1, 1,r, 7,m,WPF_new, policy_functions_w)
            store_pf(r, 1, 1, 1, 1,r, 7,m,VF, value_functions )
            store_pf(r, 1, 1, 1, 1,r, 7,m,WF, value_functions_w)

        end
    end
end

In [11]:
# next time use old version as starting point
model = model_gen()

# vfi_inputs(model, inc_input, t_in_amount, t_out_amount, t_in_prob, t_out_prob, surv_input, m, n, use_interp;
@inbounds for n in 1:7, m in 0:1
     r= 2

     # provide 
    (; VF, PF_new, WF, WPF_new) = vfi_inputs(model, r, r, 1, r, 1,r, m, n, true)

    store_pf(r, r, 1, r, 1,r, n,m,PF_new, policy_functions )
    store_pf(r, r, 1, r, 1,r, n,m,WPF_new, policy_functions_w)
    store_pf(r, r, 1, r, 1,r, n,m,VF, value_functions )
    store_pf(r, r, 1, r, 1,r, n,m,WF, value_functions_w)


end

Solution foundSolution foundSolution foundSolution foundSolution foundSolution foundSolution foundSolution foundSolution found

In [ ]:
# additional counterfactuals
@inbounds for r in 2, n in 1:7, m in 0:1

        # transfer in and income
        (; VF, PF_new, WF, WPF_new) = vfi_inputs(model, 1, 1, r, 1, r,r, m, n, true)

        store_pf(1, 1, r, 1, r,r, n,m,PF_new, policy_functions )
        store_pf(1, 1, r, 1, r,r, n,m,WPF_new, policy_functions_w)

        # transfer in amount only and income 
        (; VF, PF_new, WF, WPF_new) = vfi_inputs(model, 1, 1, r, r, r,r, m, n, true)

        store_pf(1, 1, r, r, r,r, n,m,PF_new, policy_functions )
        store_pf(1, 1, r, r, r,r, n,m,WPF_new, policy_functions_w)

        # transfer in probability only and income 
        (; VF, PF_new, WF, WPF_new) = vfi_inputs(model, 1, r, r, 1, r,r, m, n, true)

        store_pf(1, r, r, 1, r,r, n,m,PF_new, policy_functions )
        store_pf(1, r, r, 1, r,r, n,m,WPF_new, policy_functions_w)

        # transfer out and income 
        (; VF, PF_new, WF, WPF_new) = vfi_inputs(model, 1, r, 1, r, 1,r, m, n, true)

        store_pf(1, r, 1, r, 1,r, n,m,PF_new, policy_functions )
        store_pf(1, r, 1, r, 1,r, n,m,WPF_new, policy_functions_w)

        # transfer out probability only and income
        (; VF, PF_new, WF, WPF_new) = vfi_inputs(model, r, r, r, r, 1,r, m, n, true)

        store _pf(r, r, r, r, 1,r, n,m,PF_new, policy_functions )
        store_pf(r, r, r, r, 1,r, n,m,WPF_new, policy_functions_w)

        # transfer out amount only and income
        (; VF, PF_new, WF, WPF_new) = vfi_inputs(model, 1, r, 1, r, r,r, m, n, true)

        store_pf(1, r, 1, r, r,r, n,m,PF_new, policy_functions )
        store_pf(1, r, 1, r, r,r, n,m,WPF_new, policy_functions_w)
        
        # all transfers and income 
        (; VF, PF_new, WF, WPF_new) = vfi_inputs(model, 1, 1, 1, 1, 1,r, m, n, true)

        store_pf(1, 1, 1, 1, 1,r, n,m,PF_new, policy_functions )
        store_pf(1, 1, 1, 1, 1,r, n,m,WPF_new, policy_functions_w)

end

In [8]:
# Save to BSON
using BSON
BSON.@load "/Users/scanilang/Documents/econ/umn/family_transfers/pol_functions_new.bson" policy_functions
BSON.@load "/Users/scanilang/Documents/econ/umn/family_transfers/pol_functions_w_new.bson" policy_functions_w
BSON.@load "/Users/scanilang/Documents/econ/umn/family_transfers/val_functions_new.bson" value_functions
BSON.@load "/Users/scanilang/Documents/econ/umn/family_transfers/val_functions_w_new.bson" value_functions_w
# @load to load it later


In [ ]:
BSON.@save "/Users/scanilang/Documents/econ/umn/family_transfers/pol_functions_new.bson" policy_functions
BSON.@save "/Users/scanilang/Documents/econ/umn/family_transfers/pol_functions_w_new.bson" policy_functions_w
BSON.@save "/Users/scanilang/Documents/econ/umn/family_transfers/val_functions_new.bson" value_functions
BSON.@save "/Users/scanilang/Documents/econ/umn/family_transfers/val_functions_w_new.bson" value_functions_w
# @load to load it later

In [ ]:
BSON.@save "/Users/scanilang/Documents/econ/umn/year 3/3rd year paper/val_functions_new.bson" value_functions
BSON.@save "/Users/scanilang/Documents/econ/umn/year 3/3rd year paper/val_functions_w_new.bson" value_functions_w

In [8]:
function get_fam_size(draw, r)
    thresholds = r == 2 ? 
        [0.07909091, 0.19636364, 0.31545455, 0.39454545, 0.41727273, 0.42363636, 0.69636364, 0.79090909, 0.89909091, 0.96272727, 0.98272727] :
        [0.19467521, 0.36322690, 0.60264238, 0.69892904, 0.73005705, 0.73996597, 0.69636364, 0.96857171, 0.98778901, 0.99839856, 0.99949955]
    
    sizes = [2, 3, 4, 5, 6, 7, 1, 2, 3, 4, 5, 6]
    
    index = findfirst(t -> draw <= t, thresholds)
    return index === nothing ? 6 : sizes[index]
end


get_fam_size (generic function with 1 method)

In [9]:
# family structure
function black_fam_structure(N, seed)
    # set seed
    Random.seed!(seed)
    
    # draw 
    draws = rand(N)

    # placeholders
    fam_size = zeros(Int, N)
    mar_status = zeros(Int, N)

    # assignments
    fam_size = get_fam_size.(draws, 2)
    
    mar_status .= (draws .<= 0.42363636) .* 1

    return fam_size, mar_status
end

function white_fam_structure(N, seed)
    # set seed
    Random.seed!(seed)
    
    # draw 
    draws = rand(N)

    # placeholders
    fam_size = zeros(Int, N)
    mar_status = zeros(Int, N)

    # assignments
    fam_size = get_fam_size.(draws, 1)
    
    mar_status .= (draws .<= 0.73996597) .* 1

    return fam_size, mar_status
end

white_fam_structure (generic function with 1 method)

In [10]:
function assign_race_input(value, race_mat, N)
    value == 0 ? race_mat : fill(1, N)
end

function simulate_model_inputs(N, seed, model, inc_race_input, t_in_race_input, t_out_race_input, prob_in_race_input, prob_out_race_input, surv_race_input)
    
    # model parameters
    (; beta, rate, tax_a, gamma, co, cw, zpnts, apnts, asset_grid) = model

    # set seed
    Random.seed!(seed)

    # zgrids 
    Pimat_white, zgrid_white = Markov(zpnts, 1)
    Pimat_black, zgrid_black = Markov(zpnts, 2)

    # race distribution (for now equal distribution)
    num_white = Int(floor(N/2))
    num_black = Int(ceil(N/2))
    fam_size = zeros(Int, N)
    mar_stat = zeros(Int, N)

    race_mat = [repeat([1], num_white); repeat([2],num_black)]

    # race dependent inputs 
    inc_input_vec = assign_race_input(inc_race_input, race_mat, N)
    t_out_prob_vec = assign_race_input(prob_out_race_input, race_mat, N)
    t_in_prob_vec = assign_race_input(prob_in_race_input, race_mat, N)
    surv_input_vec = assign_race_input(surv_race_input, race_mat, N)
    t_in_amount_vec = assign_race_input(t_in_race_input, race_mat, N)
    t_out_amount_vec = assign_race_input(t_out_race_input, race_mat, N)

    # family structure distribution
    fam_size[1:num_white], mar_stat[1:num_white] = white_fam_structure(num_white, seed)
    fam_size[(num_white + 1):N], mar_stat[(num_white + 1):N] = black_fam_structure(num_black, seed+1)

    # placeholders
    cons_mat = zeros(N,16)
    z_mat = zeros(N,16)
    asset_mat = zeros(N,16) # initial condition 0
    age_mat = zeros(N,16)
    shock_out_mat = zeros(N,16)
    shock_in_mat = zeros(N,16)
    t_in_mat = zeros(N,16)
    t_out_mat = zeros(N,16)
    GT_mat = zeros(N,16)
    y_mat = zeros(N,16)
    Y_mat = zeros(N,16)
    BTY_mat = zeros(N,16)
    alive_mat = ones(N,16)
    past_in_mat = zeros(N,16)
    past_out_mat = zeros(N,16)

    # draw transfer and income shocks
    shock_in_draws = rand(N,16)
    shock_out_draws = rand(N,16)
    survival_draws = rand(N,6)

    eps_draws_white = rand(Normal(0, 0.4245223), num_white, 16)
    if inc_race_input == 1
        eps_draws_black = rand(Normal(0, 0.4245223), num_black, 16)
    else
        eps_draws_black = rand(Normal(0, 0.4434178), num_black, 16)
    end

    # precompute policy functions
    pf_grids = [policy_functions[inc_input_vec[i], t_in_amount_vec[i], t_out_amount_vec[i], t_in_prob_vec[i], t_out_prob_vec[i],
                    surv_input_vec[i], fam_size[i], mar_stat[i]] for i in 1:N]
    wpf_grids = [policy_functions_w[inc_input_vec[i], t_in_amount_vec[i], t_out_amount_vec[i], t_in_prob_vec[i], t_out_prob_vec[i],
                    surv_input_vec[i], fam_size[i], mar_stat[i]] for i in 1:N]

    # model simulation
    @inbounds for t in 2:16
        # update age
        age_mat[:, t] = age_mat[:, t-1] .+ 1 

        if t <= 5
            # update state variables
            z_mat[(1:num_white),t] = 0.872804 .* z_mat[(1:num_white),t-1] .+ eps_draws_white[:,t]
            if inc_race_input == 1
                z_mat[(num_white + 1):N,t] = 0.86144 .* z_mat[(num_white + 1):N,t-1] .+ eps_draws_black[:,t]
            else
                z_mat[(num_white + 1):N,t] = 0.872804 .* z_mat[(num_white + 1):N,t-1] .+ eps_draws_black[:,t]
            end
            log_income = g.(inc_input_vec,t-1,fam_size) + z_mat[:,t] # log income
            y_mat[:,t] = exp.(log_income) # income
            GT_mat[:,t] = GT.(t-1,y_mat[:,t],asset_mat[:,t-1],  mar_stat, fam_size, cw, co, tax_a, rate) # government trasnfers 
            BTY_mat[:,t] = y_mat[:,t] .+ rate.* asset_mat[:,t-1] .+ GT_mat[:,t] # before tax resources

            # update transfer shock
            shock_out_mat[:, t] = shock_out_draws[:, t] .< shocks_out_prob.(t_out_prob_vec, fam_size, mar_stat, t-1, log.(BTY_mat[:,t]), past_in_mat[:,t], past_out_mat[:,t])
            shock_in_mat[:, t] = shock_in_draws[:, t] .< shocks_in_prob.(t_in_prob_vec, fam_size, mar_stat, t-1, log.(BTY_mat[:,t]), past_in_mat[:,t], past_out_mat[:,t])

            # update past transfer indicator
            past_in_mat[:,t+1] =  max.(past_in_mat[:,t], shock_in_mat[:, t])
            past_out_mat[:,t+1] =  max.(past_out_mat[:,t], shock_out_mat[:, t])
        
            t_in_mat[:, t] = shock_in_mat[:, t] .* transfers_in_amount.(t_in_amount_vec,fam_size,mar_stat,t-1,BTY_mat[:,t])
            t_out_mat[:, t] = shock_out_mat[:, t] .* transfers_out_amount.(t_out_amount_vec,fam_size,mar_stat,t-1,BTY_mat[:,t])

            Y_mat[:,t] = Y_solve_inputs.(inc_input_vec, t_in_amount_vec, t_out_amount_vec,t-1,mar_stat,fam_size,shock_in_mat[:, t], 
                shock_out_mat[:, t] , z_mat[:,t], asset_mat[:,t-1], cw, co, tax_a, rate) # resources
            #Y_solve_inputs.(inc_input, t_in_amount, t_out_amount
            # asset choice
            for i in 1:N
                if inc_input_vec[i] == 1
                    itp = LinearInterpolation((1:9, zgrid_white, asset_grid, 0:1, 0:1, 0:1, 0:1), pf_grids[i], extrapolation_bc = Flat())
                elseif inc_input_vec[i] == 2
                    itp = LinearInterpolation((1:9, zgrid_black, asset_grid, 0:1, 0:1, 0:1, 0:1), pf_grids[i], extrapolation_bc = Flat())
                end
                asset_mat[i, t] = itp(t-1,z_mat[i,t],asset_mat[i,t-1], shock_in_mat[i, t], shock_out_mat[i, t], past_in_mat[i,t], past_out_mat[i,t]) 
            end
        
        elseif t <= 10
            # update income 
            z_mat[(1:num_white),t] = 0.872804 .* z_mat[(1:num_white),t-1] + eps_draws_white[:,t]
            if inc_race_input == 0
                z_mat[(num_white + 1):N,t] = 0.86144 .* z_mat[(num_white + 1):N,t-1] .+ eps_draws_black[:,t]
            else
                z_mat[(num_white + 1):N,t] = 0.872804 .* z_mat[(num_white + 1):N,t-1] .+ eps_draws_black[:,t]
            end
            log_income = g.(inc_input_vec,t-1,mar_stat .+ 1) + z_mat[:,t] # log income
            y_mat[:,t] = exp.(log_income)
            GT_mat[:,t] = GT.(t-1,y_mat[:,t],asset_mat[:,t-1],  mar_stat, mar_stat .+1, cw, co, tax_a, rate) # government transfers (in logs)
            BTY_mat[:,t] = y_mat[:,t] .+ rate.* asset_mat[:,t-1] .+ GT_mat[:,t] # before tax resources

            # update transfer shock
            shock_out_mat[:, t] = shock_out_draws[:, t] .< shocks_out_prob.(t_out_prob_vec, mar_stat .+ 1, mar_stat, t-1, log.(BTY_mat[:,t]), past_in_mat[:,t], past_out_mat[:,t])
            shock_in_mat[:, t] = shock_in_draws[:, t] .< shocks_in_prob.(t_in_prob_vec, mar_stat .+ 1, mar_stat, t-1, log.(BTY_mat[:,t]), past_in_mat[:,t], past_out_mat[:,t])

            # update past transfer indicator
            past_in_mat[:,t+1] =  max.(past_in_mat[:,t], shock_in_mat[:, t])
            past_out_mat[:,t+1] =  max.(past_out_mat[:,t], shock_out_mat[:, t])
  
            t_in_mat[:, t] = shock_in_mat[:, t] .* transfers_in_amount.(t_in_amount_vec,mar_stat .+ 1,mar_stat,t-1,BTY_mat[:,t])
            t_out_mat[:, t] = shock_out_mat[:, t] .* transfers_out_amount.(t_out_amount_vec,mar_stat .+ 1,mar_stat,t-1,BTY_mat[:,t])

            Y_mat[:,t] = Y_solve_inputs.(inc_input_vec, t_in_amount_vec, t_out_amount_vec,t-1,mar_stat,mar_stat .+ 1,
                            shock_in_mat[:, t], shock_out_mat[:, t] , z_mat[:,t], asset_mat[:,t-1], cw, co, tax_a, rate) # resources

             # asset choice
            for i in 1:N
                if inc_input_vec[i] == 1
                    itp = LinearInterpolation((1:9, zgrid_white, asset_grid, 0:1, 0:1, 0:1, 0:1), pf_grids[i], extrapolation_bc = Flat())
                elseif inc_input_vec[i] == 2
                    itp = LinearInterpolation((1:9, zgrid_black, asset_grid, 0:1, 0:1, 0:1, 0:1), pf_grids[i], extrapolation_bc = Flat())
                end
                asset_mat[i, t] = itp(t-1,z_mat[i,t],asset_mat[i,t-1], shock_in_mat[i, t], shock_out_mat[i, t], past_in_mat[i,t], past_out_mat[i,t])
            end
        else
            y_mat[:,t] = 0.4 * y_mat[:,9] # income
            GT_mat[:,t] = GT.(t-1,y_mat[:,t],asset_mat[:,t-1],  mar_stat, mar_stat .+ 1, cw, co, tax_a, rate) # government transfers 
            BTY_mat[:,t] = y_mat[:,t] .+ rate.* asset_mat[:,t-1] .+ GT_mat[:,t] # before tax resources

            # update transfer shock
            shock_out_mat[:, t] = shock_out_draws[:, t] .< shocks_out_prob.(t_out_prob_vec, mar_stat .+ 1, mar_stat, t-1, log.(BTY_mat[:,t]), past_in_mat[:,t], past_out_mat[:,t])
            shock_in_mat[:, t] = shock_in_draws[:, t] .< shocks_in_prob.(t_in_prob_vec, mar_stat .+ 1, mar_stat, t-1, log.(BTY_mat[:,t]), past_in_mat[:,t], past_out_mat[:,t])

            # update past transfer indicator
            past_in_mat[:,t+1] =  max.(past_in_mat[:,t], shock_in_mat[:, t])
            past_out_mat[:,t+1] =  max.(past_out_mat[:,t], shock_out_mat[:, t])

            t_in_mat[:, t] = shock_in_mat[:, t] .* transfers_in_amount.(t_in_amount_vec,mar_stat .+ 1,mar_stat,t-1,BTY_mat[:,t])
            t_out_mat[:, t] = shock_out_mat[:, t] .* transfers_out_amount.(t_out_amount_vec,mar_stat .+ 1,mar_stat,t-1,BTY_mat[:,t])

            #alive_mat[:,t] = survival_draws[:,t-9] .< [repeat([survival_prob[t-1,1]], num_white); repeat([survival_prob[t-1,2]],num_black)]
            Y_mat[:,t] = Y_solve_inputs.(inc_input_vec, t_in_amount_vec, t_out_amount_vec,t-1,mar_stat,mar_stat .+ 1,
                shock_in_mat[:, t], shock_out_mat[:, t] , z_mat[:,t], asset_mat[:,t-1], cw, co, tax_a, rate) # resources

             # asset choice
             for i in 1:N
                # survival shock
                if alive_mat[i,t-1] == 0
                    alive_mat[i,t] = 0
                elseif race_mat == 1
                    alive_mat[i,t] = survival_draws[i,t-9] .< survival_prob[t-1,1]
                else 
                    alive_mat[i,t] = survival_draws[i,t-9] .< survival_prob[t-1,2]
                end
                
                if inc_input_vec[i] == 1
                    itp = LinearInterpolation((1:9, zgrid_white, asset_grid, 0:1, 0:1, 0:1, 0:1), pf_grids[i], extrapolation_bc = Flat())
                elseif inc_input_vec[i] == 2
                    itp = LinearInterpolation((1:9, zgrid_black, asset_grid, 0:1, 0:1, 0:1, 0:1), pf_grids[i], extrapolation_bc = Flat())
                end
                asset_mat[i, t] = itp(t-1,z_mat[i,t],asset_mat[i,t-1], shock_in_mat[i, t], shock_out_mat[i, t], past_in_mat[i,t], past_out_mat[i,t])
            end

        end 
    end
    asset_mat[alive_mat .== 0] .= NaN
    z_mat[alive_mat .== 0] .= NaN
    y_mat[alive_mat .== 0] .= NaN
    Y_mat[alive_mat .== 0] .= NaN
    shock_in_mat[alive_mat .== 0] .= NaN
    shock_out_mat[alive_mat .== 0] .= NaN
    t_in_mat[alive_mat .== 0] .= NaN
    t_out_mat[alive_mat .== 0] .= NaN
    GT_mat[alive_mat .== 0] .= NaN
    past_in_mat[alive_mat .== 0] .= NaN
    past_out_mat[alive_mat .== 0] .= NaN

    return (; age_mat, asset_mat, z_mat, y_mat, Y_mat, shock_in_mat, shock_out_mat, t_in_mat, t_out_mat,GT_mat,past_in_mat, past_out_mat, fam_size, mar_stat)
end


simulate_model_inputs (generic function with 1 method)

In [17]:
#simulate_model_inputs(N, seed, model, inc_race_input, t_in_race_input, t_out_race_input, prob_in_race_input, prob_out_race_input, surv_race_input)
model = model_gen()

base_sim = simulate_model_inputs(10000, 80,  model, 0, 0, 0, 0, 0, 0)

# run counterfacturals
transfer_counter = simulate_model_inputs(10000, 80,  model, 0, 1, 1, 1, 1, 0)
income_counter = simulate_model_inputs(10000, 80,  model,  1, 0, 0, 0, 0, 0)
#provide_counter = simulate_model_inputs(5000, 100, model, 0, 0, 1, 0, 1, 0)
receive_counter = simulate_model_inputs(10000, 80, model, 0, 1, 0, 1, 0, 0)
#receive_amount_counter = simulate_model_inputs(10000, 175, model, 0, 1, 0, 0, 0, 0)

(age_mat = [0.0 1.0 … 14.0 15.0; 0.0 1.0 … 14.0 15.0; … ; 0.0 1.0 … 14.0 15.0; 0.0 1.0 … 14.0 15.0], asset_mat = [0.0 8.984449546457168e-10 … NaN NaN; 0.0 3.5628106473888873e-10 … NaN NaN; … ; 0.0 4.864150354784171e-10 … 7.013784462324889e-11 7.013784462324889e-11; 0.0 2.3323194390814507e-10 … NaN NaN], z_mat = [0.0 -0.5279658244936336 … NaN NaN; 0.0 0.0021036903954820847 … NaN NaN; … ; 0.0 0.2503348155433817 … 0.0 0.0; 0.0 0.021058561056288696 … NaN NaN], y_mat = [0.0 106746.98537351495 … NaN NaN; 0.0 212353.20009169658 … NaN NaN; … ; 0.0 138195.15879751122 … 187211.70620175044 187211.70620175044; 0.0 89458.12384840146 … NaN NaN], Y_mat = [0.0 46378.394160934564 … NaN NaN; 0.0 55907.71894395404 … NaN NaN; … ; 0.0 62351.872 … 19306.582793058522 19306.582793058522; 0.0 33124.432 … NaN NaN], shock_in_mat = [0.0 0.0 … NaN NaN; 0.0 0.0 … NaN NaN; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … NaN NaN], shock_out_mat = [0.0 1.0 … NaN NaN; 0.0 0.0 … NaN NaN; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … NaN NaN], t_in

In [ ]:
using Plots
using Statistics

base_sim.asset_mat[base_sim.asset_mat .< 0.001] .= 0 

plot(1:15, [nanmean(col) for col in eachcol(base_sim.asset_mat[1:5000,2:16])], label = "White HH", title = "Savings",xaxis = "Age", yaxis= "Assets")
plot!(1:15, [nanmean(col) for col in eachcol(base_sim.asset_mat[5001:10000,2:16])], label = "Black HH")

savefig("/Users/scanilang/Documents/econ/umn/family_transfers/savings_plot.png")

#plot!(1:16, [nanmean(col) for col in eachcol(transfer_counter.asset_mat[2501:5000,:])], label = "Black HH - transfers")
#plot!(1:16, [nanmean(col) for col in eachcol(income_counter.asset_mat[2501:5000,:])], label = "Black HH - income")

In [ ]:
plot(1:15, [nanmean(col) for col in eachcol(base_sim.Y_mat[1:5000,2:16])], label = "White HH", title = "Wealth",xaxis = "Age", yaxis= "Wealth")
plot!(1:15, [nanmean(col) for col in eachcol(base_sim.Y_mat[5001:10000,2:16])], label = "Black HH", title = "Wealth",xaxis = "Age", yaxis= "Wealth")
plot!(1:15, [nanmean(col) for col in eachcol(transfer_counter.Y_mat[5001:10000,2:16])], label = "Black HH - transfer", title = "Wealth over lifecycle",xaxis = "Age", yaxis= "Savings")
plot!(1:15, [nanmean(col) for col in eachcol(income_counter.Y_mat[5001:10000,2:16])], label = "Black HH - income", title = "Wealth over lifecycle",xaxis = "Age", yaxis= "Savings")

#savefig("/Users/scanilang/Documents/econ/umn/family_transfers/wealth_plot.png")


In [ ]:
#plot(1:16, [nanmean(col) for col in eachcol(base_sim.Y_mat[1:5000,:])], label = "White HH", title = "Wealth",xaxis = "Age", yaxis= "Wealth")
plot(1:15, [nanmean(col) for col in eachcol(base_sim.Y_mat[5001:10000,2:16])], label = "Black HH", title = "Wealth",xaxis = "Age", yaxis= "Wealth")
plot!(1:15, [nanmean(col) for col in eachcol(transfer_counter.Y_mat[5001:10000,2:16])], label = "Black HH - all counter", title = "Wealth over lifecycle",xaxis = "Age", yaxis= "Savings")
plot!(1:15, [nanmean(col) for col in eachcol(receive_counter.Y_mat[5001:10000,2:16])], label = "Black HH - receive counter", title = "Wealth over lifecycle",xaxis = "Age", yaxis= "Savings")
plot!(1:15, [nanmean(col) for col in eachcol(receive_amount_counter.Y_mat[5001:10000,2:16])], label = "Black HH - receive counter", title = "Wealth over lifecycle",xaxis = "Age", yaxis= "Savings")
#savefig("/Users/scanilang/Documents/econ/umn/family_transfers/wealth_counter_plot.png")


In [ ]:
received_idx = findall(all(iszero,base_sim.past_in_mat, dims = 2 )[:])
provided_idx = findall(all(iszero,base_sim.past_out_mat, dims = 2 )[:])

received_or_provided = union(received_idx, provided_idx)

transfer_white = intersect(received_or_provided, 1:5000)
notransfer_white = setdiff(1:5000, transfer_white)
transfer_black = intersect(received_or_provided, 5001:1000)
notransfer_black = setdiff(5001:10000, transfer_black)


In [ ]:
## keep those that had provided a transfer
received_idx = findall(all(iszero,base_sim.past_in_mat, dims = 2 )[:])
provided_idx = findall(all(iszero,base_sim.past_out_mat, dims = 2 )[:])

received_or_provided = union(received_idx, provided_idx)

transfer_white = intersect(received_or_provided, 1:5000)
notransfer_white = setdiff(1:5000, transfer_white)
transfer_black = intersect(received_or_provided, 5001:1000)
notransfer_black = setdiff(5001:10000, transfer_black)



plot(1:16, [nanmean(col) for col in eachcol(base_sim.Y_mat[transfer_white,:])], label = "White - transfer", title = "Wealth over lifecycle",xaxis = "Age", yaxis= "Wealth")
plot!(1:16, [nanmean(col) for col in eachcol(base_sim.Y_mat[notransfer_white,:])], label = "White - no transfer", title = "Wealth over lifecycle",xaxis = "Age", yaxis= "Wealth")
plot!(1:16, [nanmean(col) for col in eachcol(base_sim.Y_mat[transfer_black,:])], label = "Black - transfer", title = "Wealth over lifecycle",xaxis = "Age", yaxis= "Wealth")
plot!(1:16, [nanmean(col) for col in eachcol(base_sim.Y_mat[notransfer_black,:])], label = "Black - no transfer", title = "Wealth over lifecycle",xaxis = "Age", yaxis= "Wealth")




In [ ]:
base_sim.asset_mat[base_sim.asset_mat .< 0.01] .= 0 
 
plot(1:16, [nanmean(col) for col in eachcol(base_sim.asset_mat[transfer_white,:])], label = "White - transfer", title = "Savings over lifecycle",xaxis = "Age", yaxis= "Assets")
plot!(1:16, [nanmean(col) for col in eachcol(base_sim.asset_mat[notransfer_white,:])], label = "White - no transfer", title = "Savings over lifecycle",xaxis = "Age", yaxis= "Assets")
#plot!(1:16, [nanmean(col) for col in eachcol(base_sim.asset_mat[transfer_black,:])], label = "Black - transfer", title = "Savings over lifecycle",xaxis = "Age", yaxis= "Savings")
#plot!(1:16, [nanmean(col) for col in eachcol(base_sim.asset_mat[notransfer_black,:])], label = "Black - no transfer", title = "Savings over lifecycle",xaxis = "Age", yaxis= "Savings")


In [ ]:
## those that had provided and received a transfer

In [ ]:
## keep those that had received a transfer

In [ ]:
plot(1:16, [nanmean(col) for col in eachcol(base_sim.Y_mat[1:2500,:])], label = "White HH", title = "Wealth over lifecycle",xaxis = "Age", yaxis= "Savings")
plot!(1:16, [nanmean(col) for col in eachcol(base_sim.Y_mat[2501:5000,:])], label = "Black HH")
#plot!(1:16, [nanmean(col) for col in eachcol(transfer_counter.Y_mat[2501:5000,:])], label = "Black HH - transfers")
#plot!(1:16, [nanmean(col) for col in eachcol(income_counter.Y_mat[2501:5000,:])], label = "Black HH - income")

In [ ]:
plot(1:16, [nanmean(col) for col in eachcol(base_sim.y_mat[1:2500,:])], label = "White HH", title = "Income over lifecycle",xaxis = "Age", yaxis= "Savings")
plot!(1:16, [nanmean(col) for col in eachcol(base_sim.y_mat[2501:5000,:])], label = "Black HH")

In [ ]:
plot(1:15, [nanmean(col) for col in eachcol(base_sim.t_in_mat[1:5000,2:16])], label = "White HH", title = "Transfer Received",xaxis = "Age", yaxis= "Transfer Amount")
plot!(1:15, [nanmean(col) for col in eachcol(base_sim.t_in_mat[5001:10000,2:16])], label = "Black HH")

savefig("/Users/scanilang/Documents/econ/umn/family_transfers/tin_plot.png")

#plot!(1:16, [nanmean(col) for col in eachcol(transfer_counter.t_in_mat[2501:5000,:])], label = "Black HH - transfers")
#plot!(1:16, [nanmean(col) for col in eachcol(income_counter.t_in_mat[2501:5000,:])], label = "Black HH - income")

In [ ]:
plot(1:15, [nanmean(col) for col in eachcol(base_sim.shock_in_mat[1:2500,2:16])], label = "White HH", title = "Shock In over lifecycle",xaxis = "Age", yaxis= "Savings")
plot!(1:15, [nanmean(col) for col in eachcol(base_sim.shock_in_mat[2501:5000,2:16])], label = "Black HH")
#plot!(1:16, [nanmean(col) for col in eachcol(transfer_counter.shock_in_mat[2501:5000,:])], label = "Black HH - transfers")
#plot!(1:16, [nanmean(col) for col in eachcol(income_counter.shock_in_mat[2501:5000,:])], label = "Black HH - income")

savefig("/Users/scanilang/Documents/econ/umn/family_transfers/shockin_plot.png")


In [ ]:
plot(1:15, [nanmean(col) for col in eachcol(base_sim.shock_out_mat[1:5000,2:16])], label = "White HH", title = "Shock Out over lifecycle",xaxis = "Age", yaxis= "Savings")
plot!(1:15, [nanmean(col) for col in eachcol(base_sim.shock_out_mat[5001:10000,2:16])], label = "Black HH")
#plot!(1:16, [nanmean(col) for col in eachcol(transfer_counter.shock_out_mat[2501:5000,:])], label = "Black HH - transfers")
#plot!(1:16, [nanmean(col) for col in eachcol(income_counter.shock_out_mat[2501:5000,:])], label = "Black HH - income")

savefig("/Users/scanilang/Documents/econ/umn/family_transfers/shockout_plot.png")


In [ ]:
plot(1:15, [nanmean(col) for col in eachcol(base_sim.t_out_mat[1:5000,2:16])], label = "White HH", title = "Transfer Provided",xaxis = "Age", yaxis= "Transfer Amount")
plot!(1:15, [nanmean(col) for col in eachcol(base_sim.t_out_mat[5001:10000,2:16])], label = "Black HH")
#plot!(1:16, [nanmean(col) for col in eachcol(transfer_counter.t_out_mat[2501:5000,:])], label = "Black HH - transfers")
#plot!(1:16, [nanmean(col) for col in eachcol(income_counter.t_out_mat[2501:5000,:])], label = "Black HH - income")

savefig("/Users/scanilang/Documents/econ/umn/family_transfers/tout_plot.png")


In [13]:
using Plots
using Statistics
function ecdf(data)
    sorted_data = sort(data)
    n = length(data)
    y = (1:n) / n
    return sorted_data, y
end

function find_percentile(data, value)
    sorted_data = sort(data)
    n = length(data)
    
    # Find the index where the value would be inserted
    index = searchsortedfirst(sorted_data, value)
    
    # Calculate the percentile
    percentile = (index - 1) / n * 100
    
    return percentile
end

function calculate_racial_rank_gap(white_wealth, black_wealth, percentile)
    # Step 1: Calculate median and 90th percentile for Black households
    black_percentile = quantile(black_wealth, percentile)

    # Step 2: Find the percentile ranks of these values in the white wealth distribution
    white_ecdf = ecdf(white_wealth)
    black_rank = find_percentile(white_wealth, black_percentile) 

    # Step 3: Calculate the difference
    rank_gap = (percentile*100) - black_rank

    return rank_gap, black_rank
end

function nanmean(x)
    return mean(filter(!isnan, x))
end

#calculate_racial_rank_gap(filter(!isnan, Y_mat[1:2500, :]), filter(!isnan, Y_mat[2501:5000, :]), 0.50)

nanmean (generic function with 1 method)

In [ ]:
median(filter(!isnan,base_sim.Y_mat[1:5000, :])) - median(filter(!isnan,base_sim.Y_mat[5001:10000, :]))


In [ ]:
quantile(base_sim.Y_mat[1:5000, :], .50) - quantile(base_sim.Y_mat[5001:10000, :], .50)


In [ ]:
quantile(base_sim.Y_mat[1:5000, :], .50)

In [14]:
calculate_racial_rank_gap(filter(!isnan, base_sim.Y_mat[1:5000, 2:16]), filter(!isnan, base_sim.Y_mat[5001:10000, 2:16]), 0.50)

# 12.955

(12.754247798586135, 37.245752201413865)

In [16]:
base_sim.Y_mat[1:5000, 2:16]

5000×15 Matrix{Float64}:
      1.03773e5       1.1263e5   …  29748.0          NaN
  73773.2         70351.0           36536.8          NaN
  62281.8         57784.9           41486.9          NaN
  68709.3         90618.6             NaN            NaN
  42866.9         59670.1             NaN            NaN
  65973.9         62351.9        …    NaN            NaN
  66680.8        131045.0           30224.7          NaN
  66641.5         61451.2           28465.5          NaN
  85721.4         87631.1           45116.9          NaN
  52609.4         52609.4             NaN            NaN
      ⋮                          ⋱                 
  52609.4         95462.9           25535.3        24929.1
  49524.4         49259.0             NaN            NaN
 227247.0        337593.0               2.11582e5      1.67353e5
  42866.9         93416.5           30204.2        29207.7
      1.25982e5       1.70081e5  …  24088.1        24179.1
  55471.9         51801.9             NaN            N

In [18]:
calculate_racial_rank_gap(filter(!isnan, base_sim.Y_mat[1:5000, 2:16]), filter(!isnan, base_sim.Y_mat[5001:10000, 2:16]), 0.50)


(13.595558705409879, 36.40444129459012)

In [21]:
calculate_racial_rank_gap(filter(!isnan, transfer_counter.Y_mat[1:5000, 2:16]), filter(!isnan, transfer_counter.Y_mat[5001:10000, 2:16]), 0.50)


(12.195140787421032, 37.80485921257897)

In [22]:
calculate_racial_rank_gap(filter(!isnan, receive_counter.Y_mat[1:5000, 2:16]), filter(!isnan, receive_counter.Y_mat[5001:10000, 2:16]), 0.50)


(10.381323434671899, 39.6186765653281)

In [ ]:
calculate_racial_rank_gap(filter(!isnan, provide_counter.Y_mat[1:5000, 2:16]), filter(!isnan, provide_counter.Y_mat[5001:10000, 2:16]), 0.50)
